In [289]:
from toxic_comment_collection import get_dataset
get_dataset('zampieri2019')

In [290]:
# Import Necessary Libraries
import pandas as pd
import numpy as np
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import word_tokenize
import emoji
import re

In [291]:
# Import Dataset
import pandas as pd
df = pd.read_csv("./files/zampieri2019/zampieri2019en.csv", sep="\t")
df.head()

,id,text,labels
0,0,@USER She should ask a few native Americans wh...,['offensive']
1,1,@USER @USER Go home you’re drunk!!! @USER #MAG...,['offensive']
2,2,Amazon is investigating Chinese employees who ...,['none']
3,3,"@USER Someone should'veTaken"" this piece of sh...",['offensive']
4,4,@USER @USER Obama wanted liberals &amp; illega...,['none']


In [292]:
df['labels'].value_counts()

['none']         8840
['offensive']    4400
Name: labels, dtype: int64

In [293]:
len(df)

13240

In [294]:
# Identify null rows or zero in the dataset

df[df.isnull().any(axis=1)]
df.loc[(df==0).any(axis=1)]

,id,text,labels
0,0,@USER She should ask a few native Americans wh...,['offensive']


In [295]:
# Create new labels in the cleaned_dataset colunm

cleaned_dataset=['too risky', 'non risky']

for i in range(len(cleaned_data)):
    if cleaned_data[i]=='offensive':
        cleaned_data[i]='too risky'
    if cleaned_data[i]=="none":
        cleaned_data[i]= 'non risky'
print(cleaned_data)

0        too risky
1        too risky
2         ['none']
3        too risky
4         ['none']
           ...    
13235    too risky
13236     ['none']
13237    too risky
13238    too risky
13239     ['none']
Name: labels, Length: 13240, dtype: object


In [296]:
df['new_labels'] = df['labels'].replace({"['offensive']":'too risky', "['none']" : "non risky"})
df.head()

,id,text,labels,new_labels
0,0,@USER She should ask a few native Americans wh...,['offensive'],too risky
1,1,@USER @USER Go home you’re drunk!!! @USER #MAG...,['offensive'],too risky
2,2,Amazon is investigating Chinese employees who ...,['none'],non risky
3,3,"@USER Someone should'veTaken"" this piece of sh...",['offensive'],too risky
4,4,@USER @USER Obama wanted liberals &amp; illega...,['none'],non risky


In [297]:
#convert labels into numerical format as 
df['numeric_labels'] = df['new_labels'].replace({"non risky":'0',"too risky":'2'})
df.head()

,id,text,labels,new_labels,numeric_labels
0,0,@USER She should ask a few native Americans wh...,['offensive'],too risky,2
1,1,@USER @USER Go home you’re drunk!!! @USER #MAG...,['offensive'],too risky,2
2,2,Amazon is investigating Chinese employees who ...,['none'],non risky,0
3,3,"@USER Someone should'veTaken"" this piece of sh...",['offensive'],too risky,2
4,4,@USER @USER Obama wanted liberals &amp; illega...,['none'],non risky,0


Cleaning Sentence


In [298]:
print(f"how many missing value in the dataset {df.isnull().any().sum()}")

how many missing value in the dataset 0


In [299]:
def clean_text(text):

    text = re.sub('\d+','', text)
    text = re.sub('&amp;',' ',text)
    text = re.sub('URL',' ',text)
    text = re.sub('[^a-zA-Z]+', ' ', text)
    text = re.sub('/(\s\s\s*)/g', ' ',text)
    text = re.sub('https?://\S+|www\.\S+', '', text)
    text = re.sub('USER','',text)
    return text

In [300]:
df['cleaned_texts'] = df['text'].apply(clean_text)
df.head()

,id,text,labels,new_labels,numeric_labels,cleaned_texts
0,0,@USER She should ask a few native Americans wh...,['offensive'],too risky,2,She should ask a few native Americans what t...
1,1,@USER @USER Go home you’re drunk!!! @USER #MAG...,['offensive'],too risky,2,Go home you re drunk MAGA Trump
2,2,Amazon is investigating Chinese employees who ...,['none'],non risky,0,Amazon is investigating Chinese employees who ...
3,3,"@USER Someone should'veTaken"" this piece of sh...",['offensive'],too risky,2,Someone should veTaken this piece of shit to...
4,4,@USER @USER Obama wanted liberals &amp; illega...,['none'],non risky,0,Obama wanted liberals illegals to move into...


In [301]:
#instantiate lemmatizer
lemmatizer = WordNetLemmatizer()

In [302]:
import nltk
nltk.download('stopwords')
nltk.download('wordnet')

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/naveenkhajuria/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     /Users/naveenkhajuria/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [303]:
#select only the cleaned_texts column
texts = df['cleaned_texts']
#convert to dataframe
texts = pd.DataFrame(texts)
#create empty list to hold the final output
corpus = []
#loop through each row of comments
for i in range(0, len(texts)):
    #tokenize the text
    review = texts['cleaned_texts'][i].split()
    #remove stopwords and lemmatize
    text_stopwords_lemmatized = [lemmatizer.lemmatize(word) for word in review if not word in stopwords.words('english')]
    #combine the tokenized texts
    text_stopwords_lemmatized = ' '.join(text_stopwords_lemmatized)
    #put final result into the corpus list
    corpus.append(text_stopwords_lemmatized)

In [304]:
#create dataframe with the appropriate name as accoridng to repo guidelines
corpus = pd.DataFrame(corpus, columns=['clean_sentence_training'])

In [305]:
#merge the main dataframe with the cleaned one
df = pd.concat([df, corpus], axis=1)

In [306]:
#function to check for emojis
def extract_emojis(s):
    """Checks if there are any emojis present

    Args:
        s (pandas series): column containing the texts
    Returns:
        result (list) : list containing emoji values if present, if not, empty string
    """    
    result= ''.join(c for c in s if c in emoji.EMOJI_DATA)
    return result

In [307]:
extract_emojis(df['text'])

''

In [308]:
#rename columns to match the appropriate format requested in the repo
df = df.rename(columns={'cleaned_texts':'raw_sentence','numeric_labels':'label'})
df['corpus_name'] = 'zampieri2019: Offensive Posts in Social Media'
df['clean_sentence_EDA'] = df['clean_sentence_training']


In [309]:
#select only relevant columns
df = df[['corpus_name','raw_sentence','label','clean_sentence_training','clean_sentence_EDA']]

In [311]:
df.to_csv("zampieri2019_offensive_Posts_in_Social_Media.csv")